Scrape weather data from the website "http://www.estesparkweather.net/archive_reports.php?date=200901"
Scrape all the available attributes of weather data for each day from 2009-01-01 to 2018-10-31
Ignore records for missing days
Represent the scraped data as pandas dataframe object.

### Dataframe specific deatails
- Expected column names (order dose not matter):   
       ['Average temperature (°F)', 'Average humidity (%)',
       'Average dewpoint (°F)', 'Average barometer (in)',
       'Average windspeed (mph)', 'Average gustspeed (mph)',
       'Average direction (°deg)', 'Rainfall for month (in)',
       'Rainfall for year (in)', 'Maximum rain per minute',
       'Maximum temperature (°F)', 'Minimum temperature (°F)',
       'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
       'Minimum pressure', 'Maximum windspeed (mph)',
       'Maximum gust speed (mph)', 'Maximum heat index (°F)']
- Each record in the dataframe corresponds to weather deatils of a given day
- Make sure the index column is **date-time format (yyyy-mm-dd)**
- Perform necessary data cleaning and type cast each attributes to relevent data type

### Saving the dataframe
- Once you are done with you scrapping save your dataframe as pickle file by name 'dataframe.pk'

In [1]:
import bs4
from bs4 import BeautifulSoup
import csv
import requests
import time
import pandas as pd
import urllib
import re as rgx
import pickle

In [2]:
FROM_YEAR = 2009
TO_YEAR = 2018
FROM_MONTH = 1
TO_MONTH = 12
TO_DATE = 28

In [3]:
cell = []
for year in range(FROM_YEAR, TO_YEAR+1):
    if year == 2018:
        TO_MONTH = 10
    for month in range(FROM_MONTH, TO_MONTH+1):
        # geting data from website using requests
        # for each month there is seperate web page
        url = "https://www.estesparkweather.net/archive_reports.php?date="+ str(year) + str(month).zfill(2)
        re = requests.get(url)
        soup = BeautifulSoup(re.content, "html.parser")
        # finding all table tags on webpage and put in lst_tables variable
        lst_tables = soup.findAll("table")
        # delete last table sincce it doesnt have required info 
        del lst_tables[-1]
        td = []
        # appending each table data tag to td1 list for each table and then append td1 list to td list as nested list 
        for i in lst_tables:
            td1 = []
            var = i.findAll("td")
            for j in var:
        #         print(j.text)
                td1.append(j.text)
            td.append(td1)
        # delete last 8 td lists entries because they are not useful
        del td[-1:-9:-1]
        # delete every odd element from each nested list since it is title of colomns  
        for i in td:
            del i[1::2]
        # converting first element of each sublist to date "Jan 1 Average and Extremes" to "Jan-1-2009"
        for lst in td:
    #     print(lst[0].split()[1])
            lst[0] = lst[0].split()[0] +"-"+ lst[0].split()[1] +"-"+ str(year)
        # append every element of td to cell
        for i in td:
            cell.append(i)

In [4]:
cell

[['Jan-1-2009',
  ' 37.8 °F',
  ' 35 %',
  ' 12.7 °F',
  ' 29.7 in.',
  ' 26.4 mph',
  ' 36.8 mph',
  ' 274 ° ( W )',
  ' 0.00 in.',
  ' 0.00 in.',
  ' 0.00 in. on day 02 at time 00:00',
  ' 40.1 °F on day 01 at time 13:48',
  ' 34.5 °F on day 01 at time 03:45',
  ' 44 % on day 01 at time 22:37',
  ' 27 % on day 00 at time 00:28',
  ' 29.762  in. on day 01 at time 09:14',
  ' 29.596  in. on day 01 at time 01:29',
  ' 41.4 mph on day 01 at time 11:27',
  ' 59 mph  from 270 °( W )  on day 01 at time 21:15',
  ' 40.1 °F on day 01 at time 13:48'],
 ['Jan-2-2009',
  ' 43.2 °F',
  ' 32 %',
  ' 14.7 °F',
  ' 29.5 in.',
  ' 12.8 mph',
  ' 18.0 mph',
  ' 240 ° (WSW)',
  ' 0.00 in.',
  ' 0.00 in.',
  ' 0.00 in. on day 02 at time 23:57',
  ' 52.8 °F on day 02 at time 12:37',
  ' 37.5 °F on day 01 at time 00:50',
  ' 43 % on day 01 at time 00:15',
  ' 16 % on day 02 at time 12:13',
  ' 29.669  in. on day 01 at time 00:44',
  ' 29.268  in. on day 02 at time 23:44',
  ' 35.7 mph on day 02 at time 01

In [5]:
len(cell)

3291

In [6]:
# There are 20 colomns and if table data exceeds 20 colomns then remove that sub-list from cell
for i in range(0,len(cell)):
    if len(cell[i])>20:
        del cell[i]

IndexError: list index out of range

In [7]:
len(cell)

3283

In [8]:
# convrting cell to dataframe "df"
df = pd.DataFrame(cell, columns= ["date" , 'Average temperature (°F)', 'Average humidity (%)',
       'Average dewpoint (°F)', 'Average barometer (in)',
       'Average windspeed (mph)', 'Average gustspeed (mph)',
       'Average direction (°deg)', 'Rainfall for month (in)',
       'Rainfall for year (in)', 'Maximum rain per minute',
       'Maximum temperature (°F)', 'Minimum temperature (°F)',
       'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
       'Minimum pressure', 'Maximum windspeed (mph)',
       'Maximum gust speed (mph)', 'Maximum heat index (°F)'])

In [9]:
df.head()

,date,Average temperature (°F),Average humidity (%),Average dewpoint (°F),Average barometer (in),Average windspeed (mph),Average gustspeed (mph),Average direction (°deg),Rainfall for month (in),Rainfall for year (in),Maximum rain per minute,Maximum temperature (°F),Minimum temperature (°F),Maximum humidity (%),Minimum humidity (%),Maximum pressure,Minimum pressure,Maximum windspeed (mph),Maximum gust speed (mph),Maximum heat index (°F)
0,Jan-1-2009,37.8 °F,35 %,12.7 °F,29.7 in.,26.4 mph,36.8 mph,274 ° ( W ),0.00 in.,0.00 in.,0.00 in. on day 02 at time 00:00,40.1 °F on day 01 at time 13:48,34.5 °F on day 01 at time 03:45,44 % on day 01 at time 22:37,27 % on day 00 at time 00:28,29.762 in. on day 01 at time 09:14,29.596 in. on day 01 at time 01:29,41.4 mph on day 01 at time 11:27,59 mph from 270 °( W ) on day 01 at time 21:15,40.1 °F on day 01 at time 13:48
1,Jan-2-2009,43.2 °F,32 %,14.7 °F,29.5 in.,12.8 mph,18.0 mph,240 ° (WSW),0.00 in.,0.00 in.,0.00 in. on day 02 at time 23:57,52.8 °F on day 02 at time 12:37,37.5 °F on day 01 at time 00:50,43 % on day 01 at time 00:15,16 % on day 02 at time 12:13,29.669 in. on day 01 at time 00:44,29.268 in. on day 02 at time 23:44,35.7 mph on day 02 at time 01:55,51 mph from 270 °( W ) on day 02 at time 06:34,52.8 °F on day 02 at time 12:37
2,Jan-3-2009,25.7 °F,60 %,12.7 °F,29.7 in.,8.3 mph,12.2 mph,290 ° (WNW),0.00 in.,0.00 in.,0.00 in. on day 03 at time 23:57,41.2 °F on day 02 at time 00:11,6.7 °F on day 03 at time 23:57,89 % on day 03 at time 06:54,35 % on day 03 at time 02:56,30.232 in. on day 03 at time 23:29,29.260 in. on day 03 at time 01:29,25.3 mph on day 03 at time 02:44,38 mph from 248 °(WSW) on day 02 at time 00:05,41.2 °F on day 02 at time 00:11
3,Jan-4-2009,9.3 °F,67 %,0.1 °F,30.4 in.,2.9 mph,4.5 mph,47 ° ( NE),0.00 in.,0.00 in.,0.00 in. on day 04 at time 23:57,19.4 °F on day 04 at time 23:03,-0.0 °F on day 04 at time 21:00,79 % on day 04 at time 22:12,35 % on day 04 at time 23:19,30.566 in. on day 04 at time 09:59,30.227 in. on day 03 at time 00:00,12.7 mph on day 04 at time 23:55,20 mph from 248 °(WSW) on day 04 at time 23:33,32.0 °F on day 00 at time 00:00
4,Jan-5-2009,23.5 °F,30 %,-5.3 °F,29.9 in.,16.7 mph,23.1 mph,265 ° ( W ),0.00 in.,0.00 in.,0.00 in. on day 05 at time 23:57,30.3 °F on day 05 at time 13:27,15.1 °F on day 04 at time 02:50,56 % on day 05 at time 20:38,13 % on day 05 at time 04:16,30.233 in. on day 04 at time 00:00,29.568 in. on day 05 at time 16:14,38.0 mph on day 05 at time 13:45,53 mph from 248 °(WSW) on day 05 at time 14:33,32.0 °F on day 00 at time 00:00


In [11]:
df

,date,Average temperature (°F),Average humidity (%),Average dewpoint (°F),Average barometer (in),Average windspeed (mph),Average gustspeed (mph),Average direction (°deg),Rainfall for month (in),Rainfall for year (in),Maximum rain per minute,Maximum temperature (°F),Minimum temperature (°F),Maximum humidity (%),Minimum humidity (%),Maximum pressure,Minimum pressure,Maximum windspeed (mph),Maximum gust speed (mph),Maximum heat index (°F)
0,Jan-1-2009,37.8 °F,35 %,12.7 °F,29.7 in.,26.4 mph,36.8 mph,274 ° ( W ),0.00 in.,0.00 in.,0.00 in. on day 02 at time 00:00,40.1 °F on day 01 at time 13:48,34.5 °F on day 01 at time 03:45,44 % on day 01 at time 22:37,27 % on day 00 at time 00:28,29.762 in. on day 01 at time 09:14,29.596 in. on day 01 at time 01:29,41.4 mph on day 01 at time 11:27,59 mph from 270 °( W ) on day 01 at time 21:15,40.1 °F on day 01 at time 13:48
1,Jan-2-2009,43.2 °F,32 %,14.7 °F,29.5 in.,12.8 mph,18.0 mph,240 ° (WSW),0.00 in.,0.00 in.,0.00 in. on day 02 at time 23:57,52.8 °F on day 02 at time 12:37,37.5 °F on day 01 at time 00:50,43 % on day 01 at time 00:15,16 % on day 02 at time 12:13,29.669 in. on day 01 at time 00:44,29.268 in. on day 02 at time 23:44,35.7 mph on day 02 at time 01:55,51 mph from 270 °( W ) on day 02 at time 06:34,52.8 °F on day 02 at time 12:37
2,Jan-3-2009,25.7 °F,60 %,12.7 °F,29.7 in.,8.3 mph,12.2 mph,290 ° (WNW),0.00 in.,0.00 in.,0.00 in. on day 03 at time 23:57,41.2 °F on day 02 at time 00:11,6.7 °F on day 03 at time 23:57,89 % on day 03 at time 06:54,35 % on day 03 at time 02:56,30.232 in. on day 03 at time 23:29,29.260 in. on day 03 at time 01:29,25.3 mph on day 03 at time 02:44,38 mph from 248 °(WSW) on day 02 at time 00:05,41.2 °F on day 02 at time 00:11
3,Jan-4-2009,9.3 °F,67 %,0.1 °F,30.4 in.,2.9 mph,4.5 mph,47 ° ( NE),0.00 in.,0.00 in.,0.00 in. on day 04 at time 23:57,19.4 °F on day 04 at time 23:03,-0.0 °F on day 04 at time 21:00,79 % on day 04 at time 22:12,35 % on day 04 at time 23:19,30.566 in. on day 04 at time 09:59,30.227 in. on day 03 at time 00:00,12.7 mph on day 04 at time 23:55,20 mph from 248 °(WSW) on day 04 at time 23:33,32.0 °F on day 00 at time 00:00
4,Jan-5-2009,23.5 °F,30 %,-5.3 °F,29.9 in.,16.7 mph,23.1 mph,265 ° ( W ),0.00 in.,0.00 in.,0.00 in. on day 05 at time 23:57,30.3 °F on day 05 at time 13:27,15.1 °F on day 04 at time 02:50,56 % on day 05 at time 20:38,13 % on day 05 at time 04:16,30.233 in. on day 04 at time 00:00,29.568 in. on day 05 at time 16:14,38.0 mph on day 05 at time 13:45,53 mph from 248 °(WSW) on day 05 at time 14:33,32.0 °F on day 00 at time 00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3278,Oct-27-2018,53.3°F,37%,26.6°F,29.8 in.,10.3 mph,14.8 mph,282° (WNW),0.85 in.,6.86 in.,0.00 in. on day 27 at time 23:59,64.5°F on day 27 at time 15:13,38.3°F on day 27 at time 23:59,78% on day 27 at time 23:59,24% on day 27 at time 16:39,29.935 in. on day 27 at time 23:59,29.650 in. on day 27 at time 15:11,28.8 mph on day 27 at time 06:40,38.0 mph from 283 °(WNW) on day 27 at time 0...,64.5°F on day 27 at time 15:13
3279,Oct-28-2018,49.4°F,48%,25.2°F,29.9 in.,5.3 mph,7.6 mph,282° (WNW),0.85 in.,6.86 in.,0.00 in. on day 28 at time 23:59,67.3°F on day 28 at time 13:57,30.9°F on day 28 at time 05:41,90% on day 28 at time 06:15,19% on day 28 at time 13:57,30.040 in. on day 28 at time 07:46,29.663 in. on day 28 at time 23:59,18.4 mph on day 28 at time 13:13,25.3 mph from 260 °( W ) on day 28 at time 1...,67.3°F on day 28 at time 13:57
3280,Oct-29-2018,56.1°F,21%,16.0°F,29.6 in.,8.5 mph,12.4 mph,282° (WNW),0.86 in.,6.87 in.,0.01 in. on day 29 at time 23:07,66.8°F on day 29 at time 14:45,40.3°F on day 29 at time 23:59,59% on day 29 at time 23:59,13% on day 29 at time 17:15,29.731 in. on day 29 at time 08:23,29.402 in. on day 29 at time 15:01,26.5 mph on day 29 at time 12:20,39.1 mph from 280 °( W ) on day 29 at time 1...,66.8°F on day 29 at time 14:45
3281,Oct-30-2018,31.5°F,87%,28.2°F,29.9 in.,1.9 mph,3.1 mph,275° ( W ),0.9

In [12]:
# using regex pattern to extract float values only
pattern = r"[0-9]+[.]*[0-9]*"
for i in df:
    if i == "date":
        continue
    for j in range(len(df[i])):
        o = rgx.search(pattern, str(df[i][j]))
        df[i][j] = float(o.group())
#             print(o.group())

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3283 entries, 0 to 3282
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   date                      3283 non-null   object
 1   Average temperature (°F)  3283 non-null   object
 2   Average humidity (%)      3283 non-null   object
 3   Average dewpoint (°F)     3283 non-null   object
 4   Average barometer (in)    3283 non-null   object
 5   Average windspeed (mph)   3283 non-null   object
 6   Average gustspeed (mph)   3283 non-null   object
 7   Average direction (°deg)  3283 non-null   object
 8   Rainfall for month (in)   3283 non-null   object
 9   Rainfall for year (in)    3283 non-null   object
 10  Maximum rain per minute   3283 non-null   object
 11  Maximum temperature (°F)  3283 non-null   object
 12  Minimum temperature (°F)  3283 non-null   object
 13  Maximum humidity (%)      3283 non-null   object
 14  Minimum humidity (%)    

In [14]:
df

,date,Average temperature (°F),Average humidity (%),Average dewpoint (°F),Average barometer (in),Average windspeed (mph),Average gustspeed (mph),Average direction (°deg),Rainfall for month (in),Rainfall for year (in),Maximum rain per minute,Maximum temperature (°F),Minimum temperature (°F),Maximum humidity (%),Minimum humidity (%),Maximum pressure,Minimum pressure,Maximum windspeed (mph),Maximum gust speed (mph),Maximum heat index (°F)
0,Jan-1-2009,37.8,35.0,12.7,29.7,26.4,36.8,274.0,0.0,0.0,0.0,40.1,34.5,44.0,27.0,29.762,29.596,41.4,59.0,40.1
1,Jan-2-2009,43.2,32.0,14.7,29.5,12.8,18.0,240.0,0.0,0.0,0.0,52.8,37.5,43.0,16.0,29.669,29.268,35.7,51.0,52.8
2,Jan-3-2009,25.7,60.0,12.7,29.7,8.3,12.2,290.0,0.0,0.0,0.0,41.2,6.7,89.0,35.0,30.232,29.26,25.3,38.0,41.2
3,Jan-4-2009,9.3,67.0,0.1,30.4,2.9,4.5,47.0,0.0,0.0,0.0,19.4,0.0,79.0,35.0,30.566,30.227,12.7,20.0,32.0
4,Jan-5-2009,23.5,30.0,5.3,29.9,16.7,23.1,265.0,0.0,0.0,0.0,30.3,15.1,56.0,13.0,30.233,29.568,38.0,53.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3278,Oct-27-2018,53.3,37.0,26.6,29.8,10.3,14.8,282.0,0.85,6.86,0.0,64.5,38.3,78.0,24.0,29.935,29.65,28.8,38.0,64.5
3279,Oct-28-2018,49.4,48.0,25.2,29.9,5.3,7.6,282.0,0.85,6.86,0.0,67.3,30.9,90.0,19.0,30.04,29.663,18.4,25.3,67.3
3280,Oct-29-2018,56.1,21.0,16.0,29.6,8.5,12.4,282.0,0.86,6.87,0.01,66.8,40.3,59.0,13.0,29.731,29.402,26.5,39.1,66.8
3281,Oct-30-2018,31.5,87.0,28.2,29.9,1.9,3.1,275.0,0.99,7.0,0.01,40.3,27.8,92.0,60.0,30.121,29.623,9.2,16.1,40.2


In [15]:
import datetime

In [16]:
df['date'] = pd.to_datetime(df['date'])

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3283 entries, 0 to 3282
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   date                      3283 non-null   datetime64[ns]
 1   Average temperature (°F)  3283 non-null   object        
 2   Average humidity (%)      3283 non-null   object        
 3   Average dewpoint (°F)     3283 non-null   object        
 4   Average barometer (in)    3283 non-null   object        
 5   Average windspeed (mph)   3283 non-null   object        
 6   Average gustspeed (mph)   3283 non-null   object        
 7   Average direction (°deg)  3283 non-null   object        
 8   Rainfall for month (in)   3283 non-null   object        
 9   Rainfall for year (in)    3283 non-null   object        
 10  Maximum rain per minute   3283 non-null   object        
 11  Maximum temperature (°F)  3283 non-null   object        
 12  Minimum temperature 

In [18]:
# setting index to date
df = df.set_index('date')

In [27]:
# covet data type to float

In [25]:
df = df.astype(dtype="float64")

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3283 entries, 2009-01-01 to 2018-10-31
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Average temperature (°F)  3283 non-null   float64
 1   Average humidity (%)      3283 non-null   float64
 2   Average dewpoint (°F)     3283 non-null   float64
 3   Average barometer (in)    3283 non-null   float64
 4   Average windspeed (mph)   3283 non-null   float64
 5   Average gustspeed (mph)   3283 non-null   float64
 6   Average direction (°deg)  3283 non-null   float64
 7   Rainfall for month (in)   3283 non-null   float64
 8   Rainfall for year (in)    3283 non-null   float64
 9   Maximum rain per minute   3283 non-null   float64
 10  Maximum temperature (°F)  3283 non-null   float64
 11  Minimum temperature (°F)  3283 non-null   float64
 12  Maximum humidity (%)      3283 non-null   float64
 13  Minimum humidity (%)      3283 non-null   flo

In [28]:
# saving as pickle data

In [29]:
import pickle
with open("dataframe.pk", "wb") as file:
    pickle.dump(df, file)